Scrapes the novelupdates webpage, http://www.novelupdates.com/
for novel information and cleans it.
Finally saves the dataset as a csv file.

Didn't find any easy way to get all the novel ids, hence, the novels listing is first used to get all ids.

In [37]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import re

novels_page = "http://www.novelupdates.com/novelslisting/?st=1&pg="

In [40]:
# Get the maximum number of novel pages from the listings

def get_novel_list_max_pages(page):
    soup = BeautifulSoup(page.content, 'html.parser')
    dig_pag = soup.find('div', attrs={'class':'digg_pagination'})
    page_links = dig_pag.find_all('a')
    last_page_link = str(page_links[2]) # The last page is the 3rd
    num = re.search('pg=\d+', last_page_link).group()[3:]
    return int(num)

page = requests.get(novels_page + '1')
novels_max_pages = get_novel_list_max_pages(page)

In [41]:
# Get all novel ids from the novels listing

# Overwrite for testing
# novels_max_pages = 3 

def get_novel_ids(page):
    soup = BeautifulSoup(page.content, 'html.parser')
    table = soup.find('table', attrs={'id':'myTable'})
    table_body = table.find('tbody')
    rows = table_body.find_all('tr')
    
    novel_ids = []
    for row in rows:
        col = row.find_all('td')[-1]
        novel_id = col.a['id'][3:]
        novel_ids.append(novel_id)
    return novel_ids

all_novel_ids = []
for i in range(1,novels_max_pages+1):
    page = requests.get(novels_page + str(i))
    novel_ids = get_novel_ids(page)
    all_novel_ids.extend(novel_ids)
    time.sleep(1)

df = pd.DataFrame(all_novel_ids)
print(df.head)
df.to_csv("novel_ids.csv")

<bound method NDFrame.head of           0
0      1173
1      1175
2      1177
3     13038
4      1171
5      2042
6      6612
7     11133
8      9343
9      8233
10     7959
11     2696
12    13155
13     2742
14     7145
15     5400
16     9314
17    10927
18     1179
19    12408
20    13080
21     1069
22    13197
23    12584
24     1181
25     1183
26     3992
27     1185
28     8674
29    10925
...     ...
2952   8640
2953  10129
2954    567
2955   5306
2956   1916
2957   4230
2958   6391
2959  12628
2960   2029
2961   5992
2962    364
2963   5402
2964   2031
2965   5585
2966   2033
2967   2035
2968   2037
2969    177
2970   2039
2971   2916
2972  11539
2973   8591
2974   6871
2975   5205
2976   7143
2977   8059
2978   9480
2979  10040
2980   5519
2981  11601

[2982 rows x 1 columns]>
